# Solutions to Ipywidgets exercises

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook

## 1. Dynamically disabling and hiding widgets

In [ ]:
rb = widgets.RadioButtons(
    options=["1D", "2D", "3D"],
    description='Ndims:',
    value="1D")

ndim = int(rb.value[0])

pos = []
vel = []
for i, x in enumerate("xyz"):
    pos.append(widgets.FloatText(value=0, description="{}:".format(x), disabled=ndim<1+i))
    vel.append(widgets.FloatText(value=0, description="v_{}:".format(x)))
    if i > ndim-1:
        vel[-1].layout.visibility = 'hidden'

def update_components(change):
    ndim = int(change["new"][0])
    for i in range(len(pos)):
        pos[i].disabled = ndim < 1+i
        if i > ndim-1:
            vel[i].layout.visibility = 'hidden'
        else:
            vel[i].layout.visibility = 'visible'

rb.observe(update_components, names="value")

box = widgets.HBox([rb, widgets.VBox(pos), widgets.VBox(vel)])
display(box)

**BONUS:**

In [ ]:
ndim = widgets.IntText(value=1, description='Ndims:')

pos = []
vel = []
for i in range(ndim.value):
    pos.append(widgets.FloatText(value=0, description="pos_{}:".format(i)))
    vel.append(widgets.FloatText(value=0, description="vel_{}:".format(i)))

pos_box = widgets.VBox(pos)
vel_box = widgets.VBox(vel)

def update_components(change):
    ndim_new = change["new"]
    ndim_old = len(pos_box.children)
    if ndim_old < ndim_new:
        new_pos = []
        new_vel = []
        for i in range(ndim_new-ndim_old):
            new_pos.append(widgets.FloatText(value=0, description="pos_{}:".format(i + ndim_old)))
            new_vel.append(widgets.FloatText(value=0, description="vel_{}:".format(i + ndim_old)))
        pos_box.children = list(pos_box.children) + new_pos
        vel_box.children = list(vel_box.children) + new_vel
    else:
        pos_box.children = [pos_box.children[i] for i in range(ndim_new)]
        vel_box.children = [vel_box.children[i] for i in range(ndim_new)]

ndim.observe(update_components, names="value")

box = widgets.HBox([ndim, pos_box, vel_box])
display(box)

## 2. Use a slider to change the number of scatter points in a Matplotlib figure

In [ ]:
N = 100
x = np.random.normal(0.0, scale=20.0, size=N)
y = np.random.normal(0.0, scale=20.0, size=N)

fig, ax = plt.subplots()
scat = ax.scatter(x, y, color="blue", alpha=0.5)

cp = widgets.ColorPicker(
    concise=False,
    description='Pick a color',
    value='blue',
    disabled=False
)

def update_scatter(change):
    ax.collections = []
    M = int(change["new"])
    x = np.random.normal(0.0, scale=20.0, size=M)
    y = np.random.normal(0.0, scale=20.0, size=M)
    ax.scatter(x, y, color=cp.value, alpha=0.5)

sl = widgets.FloatLogSlider(
    value=N,
    base=10,
    min=0, # max exponent of base
    max=5, # min exponent of base
    step=0.1, # exponent step
    description='Log Slider'
)

sl.observe(update_scatter, names="value")

def update_color(change):
    ax.collections[-1].set_color(change["new"])

cp.observe(update_color, names="value")

display(widgets.HBox([sl, cp]))

## 3. Create an interface to dynamically resize an image

In [ ]:
import PIL.Image as Image

img = Image.open("../../docs/images/ESS_02_small.jpg")

im_size = [img.width, img.height]

width = widgets.IntText(value=img.width, description='Width:')
height = widgets.IntText(value=img.height, description='Height:')
button = widgets.Button(description='Resize!')

hbox = widgets.HBox([width, height, button])
im_widget = widgets.Image(value=img._repr_png_(), width=img.width, height=img.height)

def resize_image(event):
    resized = img.copy().resize((width.value, height.value))
#     resized = resized.resize((im_size[0], im_size[1]))
    im_widget.value = resized._repr_png_()
    im_widget.width = width.value
    im_widget.height = height.value
    return

button.on_click(resize_image)

sl_width = widgets.IntSlider(value=img.width, min=2, max=img.width)
sl_height = widgets.IntSlider(value=img.height, min=2, max=img.height)

def update_width(change):
    width.value = change["new"]
    resize_image(None)
    return

def update_height(change):
    height.value = change["new"]
    resize_image(None)
    return

sl_width.observe(update_width, names="value")
sl_height.observe(update_height, names="value")

display(widgets.VBox([im_widget, hbox]))
display(widgets.HBox([sl_width, sl_height]))